# Análisis de Ventas con PySpark
Este notebook realiza análisis de datos de ventas usando Spark

In [ ]:
# Importar configuraciones
import sys
import os
sys.path.append(os.path.abspath('./src'))
os.makedirs("resultados", exist_ok=True)

from config.spark_config import SparkSession
from etl.transformaciones import TransformacionesVentas
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
spark = SparkSession.getActiveSession()
if spark:
    spark.stop()
    
# Inicializar Spark
spark = SparkSession.builder \
    .appName("AnalisisVentas-Notebook") \
    .enableHiveSupport() \
    .getOrCreate()
transformaciones = TransformacionesVentas(spark)
spark.conf.get("spark.sql.catalogImplementation")

# Cargar datos
ventas_df, productos_df = transformaciones.cargar_datos()

print("📊 Vista previa de ventas:")
ventas_df.show(5)

print("\n📦 Vista previa de productos:")
productos_df.show(5)

print(f"\nTotal ventas: {ventas_df.count()}")
print(f"Total productos: {productos_df.count()}")

# Esquema de datos
print("Esquema ventas:")
ventas_df.printSchema()

print("\nEsquema productos:")
productos_df.printSchema()

# Calcular métricas
ventas_completas_df, metricas_df = transformaciones.calcular_metricas(ventas_df, productos_df)

print("📈 Métricas por categoría:")
metricas_df.show()

# Análisis temporal
analisis_temporal_df = transformaciones.analisis_temporal(ventas_df)

print("📅 Análisis por fecha:")
analisis_temporal_df.show()


# Top clientes
top_clientes_df = transformaciones.top_clientes(ventas_df, top_n=5)

print("👑 Top 5 clientes:")
top_clientes_df.show()

# Análisis adicional: Ventas por producto
ventas_por_producto = ventas_completas_df.groupBy("nombre", "categoria") \
    .agg(
        F.sum("cantidad").alias("unidades_vendidas"),
        F.sum("venta_total").alias("ingresos"),
        F.round(F.avg("precio_unitario"), 2).alias("precio_promedio")
    ) \
    .orderBy(F.desc("ingresos"))

print("🏆 Productos más vendidos:")
ventas_por_producto.show()

# Visualización (si matplotlib está disponible)
try:
    import matplotlib.pyplot as plt
    
    # Convertir a pandas para visualización
    metricas_pandas = metricas_df.toPandas()
    
    plt.figure(figsize=(10, 6))
    plt.bar(metricas_pandas['categoria'], metricas_pandas['ingresos_totales'])
    plt.title('Ingresos por Categoría')
    plt.xlabel('Categoría')
    plt.ylabel('Ingresos Totales')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib no disponible para visualización")

# Esquema y Contenido
metricas_df.printSchema()
metricas_df.show()

# Modo no profesional
# Guardar resultados
print("💾 Guardando resultados...")

# Guardar como Parquet (formato optimizado para Spark)
metricas_df.write.mode("overwrite").parquet("file:///home/hadoop/proyecto-spark-vscode/resultados/metricas_ventas.parquet")
ventas_completas_df.write.mode("overwrite").parquet("file:///home/hadoop/proyecto-spark-vscode/resultados/ventas_completas.parquet")

print("✅ Resultados guardados en formato Parquet")

# Modo dinámica y portable
import os

print("💾 Guardando resultados...")

# Obtener ruta base del proyecto dinámicamente
project_root = os.getcwd()

# Carpeta resultados
output_dir = os.path.join(project_root, "resultados")

# Crear carpeta si no existe
os.makedirs(output_dir, exist_ok=True)

# Convertir a formato que Spark entiende
output_uri = f"file://{output_dir}"

# Guardar
metricas_df.write.mode("overwrite").parquet(
    f"{output_uri}/metricas_ventas.parquet"
)

ventas_completas_df.write.mode("overwrite").parquet(
    f"{output_uri}/ventas_completas.parquet"
)

print("✅ Resultados guardados en formato Parquet")

# Modo clasico en local
os.makedirs("resultados", exist_ok=True)

metricas_df.write.mode("overwrite").parquet("resultados/metricas_ventas.parquet")
ventas_completas_df.write.mode("overwrite").parquet("resultados/ventas_completas.parquet")

# Trabajar con Hive (si está configurado)
try:
    # Crear tabla Hive temporal
    ventas_completas_df.createOrReplaceTempView("ventas_completas")
    
    # Consulta SQL
    print("🔍 Consulta SQL a través de Hive:")
    resultado_sql = spark.sql("""
        SELECT categoria, 
               SUM(venta_total) as ingresos_totales,
               AVG(precio_unitario) as precio_promedio
        FROM ventas_completas
        GROUP BY categoria
        ORDER BY ingresos_totales DESC
    """)
    
    resultado_sql.show()
    
except Exception as e:
    print(f"Hive no disponible: {e}")

# Detener Spark
from config.spark_config import detener_spark_session
detener_spark_session(spark)
